## Usando starmap:

In [21]:
import numpy as np
from multiprocessing import Pool

# Tamaño del array
value = 5 * 10**7
X = np.random.rand(value)

#Nueva función de reducción con dos argumentos más
def sum_multiprocessing(A, ini, fin):
    s = 0
    for i in range(ini, fin):
        s += A[i]
    return s

def dividir_array(array, num_procesos):
    size = len(array)
    chunk_size = size // num_procesos
    ranges = []

    if num_procesos == 1:
        ranges = [(array, 0, size)]
    elif num_procesos == 2:
        ranges = [(array, 0, size // 2), (array, size//2, size)]
    elif num_procesos == 4:
        ranges = [(array, 0, size // 4), 
                  (array, size //4, size // 2), 
                  (array, size // 2, 3 * size//4), 
                  (array, 3*size // 4, size)]

    #Aplicación de starmap
    with Pool(processes=num_procesos) as pool:
        partial_sums = pool.starmap(sum_multiprocessing, ranges)

    #Suma total recoge las sumas parciales
    total_sum = sum(partial_sums)
    return total_sum

#Aplicar la función para 1, 2 y 4 procesos
for processes in [1, 2, 4]:
    tiempo = %timeit -r 2 -o dividir_array(X, processes)
    result = dividir_array(X, processes)
    print(f"\nTiempo para {processes} procesos:", tiempo, sep="\n")
    print(f"Resultado de la suma con {processes} procesos: {result}")


5.67 s ± 6.58 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)

Tiempo para 1 procesos:
5.67 s ± 6.58 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
Resultado de la suma con 1 procesos: 25000725.649850477
3.36 s ± 3.1 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)

Tiempo para 2 procesos:
3.36 s ± 3.1 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
Resultado de la suma con 2 procesos: 25000725.64985007
3.02 s ± 7.89 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)

Tiempo para 4 procesos:
3.02 s ± 7.89 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
Resultado de la suma con 4 procesos: 25000725.649849202


## De GPU a CPU:

In [36]:
import numpy as np
import cupy as cp
import time

#Crear el array en la CPU
value = 5 * 10**5
X_cpu = np.random.rand(value)

In [50]:
start = time.time()
X_gpu = cp.asarray(X_cpu) #Transferir el array a la GPU 
end = time.time()
print(f"Tiempo transfiriendo el array desde la CPU a la GPU: {end - start:.7f} segundos")
cpu_gpu = end - start


start = time.time()
suma_gpu = cp.sum(X_gpu) #Realizar la operación en la GPU
end = time.time()
print(f"Tiempo en realizar la operación desde la GPU: {end - start:.7f} segundos")
gpu_sum = end - start

start = time.time()
suma_cpu = suma_gpu.get() #Transferir el resultado de vuelta a la CPU
end = time.time()
gpu_cpu = end - start
print(f"Tiempo transfiriendo el resultado desde la GPU a la CPU: {end - start:.7f} segundos")

print(f"Tiempo total: {cpu_gpu + gpu_sum + gpu_cpu:.7f} segundos")


Tiempo transfiriendo el array desde la CPU a la GPU: 0.0283945 segundos
Tiempo en realizar la operación desde la GPU: 0.0003138 segundos
Tiempo transfiriendo el resultado desde la GPU a la CPU: 0.0000751 segundos
Tiempo total: 0.0287833 segundos


## Usando el decorador @reduce de Numba

In [61]:
# from numba import reduce

# @reduce
# def add_reduce(a, b):
#     return a + b

El tiempo que le toma a la operación sum_multiprocessing en realizarse sobre los arrays va disminuyendo conforme más procesos se involucren, como era de esperar. También se ha podido comprobar el tiempo real que le toma a la GPU hacer las operaciones cuando recibe los datos desde la CPU. 